In [24]:
import pandas as pd
import seaborn as sns
import numpy as np
from pathlib import Path
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')


Оставим и сгенерируем признаки, которые мы знаем до следующих игр, т.е если мы будем брать количество пассов на каждую игру для обучения, то при прогнозирование будущей игры мы эти данные не знаем.
Оставим сезоны начиная с 2018, 2023 уходит на валидацию, поэтому при создании признаков не учитываем этот сезон

In [25]:
n = 100 #количество матчей для валидации

pd.set_option('display.max_columns', None)

path_df = '../../../EDA/official/eda_data/games.csv'
path_club = '../../../EDA/official/eda_data/clubs_stat.csv'

df = pd.read_csv(Path(path_df).resolve())
df_club = pd.read_csv(Path(path_club).resolve())

df.head()

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_id,gameweek_gameweek,kickoff_millis,kickoff_label,provisionalKickoff_millis,provisionalKickoff_label,teams_team_1_name,teams_team_1_club_abbr,teams_team_1_club_id,teams_score_1,teams_team_2_name,teams_team_2_club_abbr,teams_team_2_club_id,teams_score_2,ground_name,ground_city,ground_id,outcome,attendance,clock_secs,clock_label,behindClosedDoors,id,team_1_formation_used,team_1_accurate_back_zone_pass,team_1_total_back_zone_pass,team_1_accurate_pass,team_1_touches,team_1_total_pass,team_1_backward_pass,team_1_successful_open_play_pass,team_1_open_play_pass,team_1_possession_percentage,team_1_leftside_pass,team_1_accurate_fwd_zone_pass,team_1_rightside_pass,team_1_total_long_balls,team_1_total_fwd_zone_pass,team_1_touches_in_opp_box,team_1_final_third_entries,team_1_crosses_18yard,team_1_successful_final_third_passes,team_1_accurate_cross,team_1_long_pass_own_to_opp,team_1_fwd_pass,team_1_total_cross_nocorner,team_1_total_cross,team_1_total_final_third_passes,team_1_long_pass_own_to_opp_success,team_1_poss_lost_all,team_1_poss_lost_ctrl,team_1_pen_area_entries,team_1_passes_left,team_1_accurate_long_balls,team_1_crosses_18yardplus,team_1_accurate_cross_nocorner,team_1_attempts_ibox,team_1_total_scoring_att,team_1_att_openplay,team_1_att_lf_target,team_1_att_bx_centre,team_1_ontarget_scoring_att,team_1_ball_recovery,team_1_att_lf_total,team_1_poss_won_att_3rd,team_1_att_ibox_target,team_1_ontarget_att_assist,team_1_blocked_pass,team_1_total_att_assist,team_1_duel_won,team_1_won_contest,team_1_att_sv_low_centre,team_1_successful_put_through,team_1_total_contest,team_1_att_assist_openplay,team_1_put_through,team_1_poss_won_mid_3rd,team_1_total_tackle,team_1_interception,team_1_interception_won,team_1_won_tackle,team_1_total_offside,team_1_duel_lost,team_1_accurate_layoffs,team_1_passes_right,team_1_total_chipped_pass,team_1_total_layoffs,team_1_accurate_chipped_pass,team_1_aerial_lost,team_1_effective_clearance,team_1_lost_corners,team_1_total_launches,team_1_total_clearance,team_1_attempts_conceded_ibox,team_1_fk_foul_won,team_1_poss_won_def_3rd,team_1_saved_ibox,team_1_saves,team_1_diving_save,team_1_challenge_lost,team_1_dispossessed,team_1_unsuccessful_touch,team_1_goal_kicks,team_1_accurate_goal_kicks,team_1_total_keeper_sweeper,team_1_attempts_conceded_obox,team_1_head_clearance,team_1_accurate_keeper_sweeper,team_1_effective_head_clearance,team_1_fk_foul_lost,team_1_attempted_tackle_foul,team_1_aerial_won,team_1_goals_conceded_ibox,team_1_goals_conceded,team_1_total_yel_card,team_1_shot_off_target,team_1_total_throws,team_1_att_rf_total,team_1_accurate_throws,team_1_att_fastbreak,team_1_att_ibox_miss,team_1_total_fastbreak,team_1_shot_fastbreak,team_1_offtarget_att_assist,team_1_att_miss_high_right,team_1_saved_obox,team_1_outfielder_block,team_1_att_rf_target,team_1_corner_taken,team_1_won_corners,team_1_att_sv_high_centre,team_1_accurate_corners_intobox,team_1_att_assist_setplay,team_1_att_miss_right,team_1_att_corner,team_1_total_corners_intobox,team_1_att_hd_total,team_1_att_hd_miss,team_1_effective_blocked_cross,team_1_blocked_cross,team_1_subs_made,team_1_accurate_launches,team_1_fouled_final_third,team_1_att_obx_centre,team_1_attempts_obox,team_1_att_obox_target,team_1_keeper_throws,team_1_accurate_keeper_throws,team_1_total_through_ball,team_1_big_chance_missed,team_1_big_chance_created,team_1_att_miss_left,team_1_accurate_through_ball,team_1_blocked_scoring_att,team_1_att_ibox_blocked,team_1_total_flick_on,team_1_losses,team_1_att_cmiss_left,team_2_formation_used,team_2_open_play_pass,team_2_leftside_pass,team_2_accurate_goal_kicks,team_2_possession_percentage,team_2_poss_lost_all,team_2_goal_kicks,team_2_poss_lost_ctrl,team_2_total_pass,team_2_total_long_balls,team_2_total_back_zone_pass,team_2_total_chipped_pass,team_2_successful_open_play_pass,team_2_accurate_long_balls,team_2_accurate_back_zone_pass,team_2_rightside_pass,team_2_unsuccessful_touch

In [26]:
home_games = df[['gameweek_compSeason_label', 'teams_team_1_name']].rename(columns={'gameweek_compSeason_label':'Season', 'teams_team_1_name': 'Team'})
away_games = df[['gameweek_compSeason_label', 'teams_team_2_name']].rename(columns={'gameweek_compSeason_label':'Season','teams_team_2_name': 'Team'})

all_games = pd.concat([home_games, away_games])
all_games['Games']=1

result = all_games.groupby(['Season', 'Team']).agg({'Games': 'sum'}).reset_index()
result

,Season,Team,Games
0,2014,Arsenal,38
1,2014,Aston Villa,38
2,2014,Burnley,38
3,2014,Chelsea,38
4,2014,Crystal Palace,38
...,...,...,...
195,2023,Nottingham Forest,22
196,2023,Sheffield United,23
197,2023,Tottenham Hotspur,23
198,2023,West Ham United,22


In [27]:
df

,match_id,gameweek_id,gameweek_compSeason_label,gameweek_compSeason_id,gameweek_gameweek,kickoff_millis,kickoff_label,provisionalKickoff_millis,provisionalKickoff_label,teams_team_1_name,teams_team_1_club_abbr,teams_team_1_club_id,teams_score_1,teams_team_2_name,teams_team_2_club_abbr,teams_team_2_club_id,teams_score_2,ground_name,ground_city,ground_id,outcome,attendance,clock_secs,clock_label,behindClosedDoors,id,team_1_formation_used,team_1_accurate_back_zone_pass,team_1_total_back_zone_pass,team_1_accurate_pass,team_1_touches,team_1_total_pass,team_1_backward_pass,team_1_successful_open_play_pass,team_1_open_play_pass,team_1_possession_percentage,team_1_leftside_pass,team_1_accurate_fwd_zone_pass,team_1_rightside_pass,team_1_total_long_balls,team_1_total_fwd_zone_pass,team_1_touches_in_opp_box,team_1_final_third_entries,team_1_crosses_18yard,team_1_successful_final_third_passes,team_1_accurate_cross,team_1_long_pass_own_to_opp,team_1_fwd_pass,team_1_total_cross_nocorner,team_1_total_cross,team_1_total_final_third_passes,team_1_long_pass_own_to_opp_success,team_1_poss_lost_all,team_1_poss_lost_ctrl,team_1_pen_area_entries,team_1_passes_left,team_1_accurate_long_balls,team_1_crosses_18yardplus,team_1_accurate_cross_nocorner,team_1_attempts_ibox,team_1_total_scoring_att,team_1_att_openplay,team_1_att_lf_target,team_1_att_bx_centre,team_1_ontarget_scoring_att,team_1_ball_recovery,team_1_att_lf_total,team_1_poss_won_att_3rd,team_1_att_ibox_target,team_1_ontarget_att_assist,team_1_blocked_pass,team_1_total_att_assist,team_1_duel_won,team_1_won_contest,team_1_att_sv_low_centre,team_1_successful_put_through,team_1_total_contest,team_1_att_assist_openplay,team_1_put_through,team_1_poss_won_mid_3rd,team_1_total_tackle,team_1_interception,team_1_interception_won,team_1_won_tackle,team_1_total_offside,team_1_duel_lost,team_1_accurate_layoffs,team_1_passes_right,team_1_total_chipped_pass,team_1_total_layoffs,team_1_accurate_chipped_pass,team_1_aerial_lost,team_1_effective_clearance,team_1_lost_corners,team_1_total_launches,team_1_total_clearance,team_1_attempts_conceded_ibox,team_1_fk_foul_won,team_1_poss_won_def_3rd,team_1_saved_ibox,team_1_saves,team_1_diving_save,team_1_challenge_lost,team_1_dispossessed,team_1_unsuccessful_touch,team_1_goal_kicks,team_1_accurate_goal_kicks,team_1_total_keeper_sweeper,team_1_attempts_conceded_obox,team_1_head_clearance,team_1_accurate_keeper_sweeper,team_1_effective_head_clearance,team_1_fk_foul_lost,team_1_attempted_tackle_foul,team_1_aerial_won,team_1_goals_conceded_ibox,team_1_goals_conceded,team_1_total_yel_card,team_1_shot_off_target,team_1_total_throws,team_1_att_rf_total,team_1_accurate_throws,team_1_att_fastbreak,team_1_att_ibox_miss,team_1_total_fastbreak,team_1_shot_fastbreak,team_1_offtarget_att_assist,team_1_att_miss_high_right,team_1_saved_obox,team_1_outfielder_block,team_1_att_rf_target,team_1_corner_taken,team_1_won_corners,team_1_att_sv_high_centre,team_1_accurate_corners_intobox,team_1_att_assist_setplay,team_1_att_miss_right,team_1_att_corner,team_1_total_corners_intobox,team_1_att_hd_total,team_1_att_hd_miss,team_1_effective_blocked_cross,team_1_blocked_cross,team_1_subs_made,team_1_accurate_launches,team_1_fouled_final_third,team_1_att_obx_centre,team_1_attempts_obox,team_1_att_obox_target,team_1_keeper_throws,team_1_accurate_keeper_throws,team_1_total_through_ball,team_1_big_chance_missed,team_1_big_chance_created,team_1_att_miss_left,team_1_accurate_through_ball,team_1_blocked_scoring_att,team_1_att_ibox_blocked,team_1_total_flick_on,team_1_losses,team_1_att_cmiss_left,team_2_formation_used,team_2_open_play_pass,team_2_leftside_pass,team_2_accurate_goal_kicks,team_2_possession_percentage,team_2_poss_lost_all,team_2_goal_kicks,team_2_poss_lost_ctrl,team_2_total_pass,team_2_total_long_balls,team_2_total_back_zone_pass,team_2_total_chipped_pass,team_2_successful_open_play_pass,team_2_accurate_long_balls,team_2_accurate_back_zone_pass,team_2_rightside_pass,team_2_unsuccessful_touch

team_1_hue - target (Выиграла команда 1 -1, проиграла -0, ничья - 2)

In [28]:
loc_home = [2, 9] + list(range(list(df.columns).index('team_1_formation_used'), list(df.columns).index('team_1_att_cmiss_left')))
loc_away = [2, 13] + list(range(list(df.columns).index('team_2_formation_used'), list(df.columns).index('team_2_att_cmiss_left')))

df_home = df.iloc[:, loc_home]
df_away = df.iloc[:, loc_away]

col_home = [i.replace('team_1_', '') for i in list(df_home.columns)]
col_away = [i.replace('team_2_', '') for i in list(df_away.columns)]

In [29]:
indx_del_home = [i for i, item in enumerate(col_home) if item not in col_away]
indx_del_away = [i for i, item in enumerate(col_away) if item not in col_home]

df_home = df_home.drop(df_home.columns[indx_del_home], axis=1)
df_away = df_away.drop(df_away.columns[indx_del_away], axis=1)

In [30]:
col_home = {i: i.replace('team_1_', '') for i in list(df_home.columns)}
col_away = {i: i.replace('team_2_', '') for i in list(df_away.columns)}

df_home.rename(columns=col_home, inplace=True)
df_away.rename(columns=col_away, inplace=True)

In [31]:
df_home = df_home.groupby(['gameweek_compSeason_label', 'teams_name']).sum().reset_index()
df_away = df_away.groupby(['gameweek_compSeason_label', 'teams_name']).sum().reset_index()

In [32]:
df_games = pd.merge(df_home, df_away, how='left', left_on=['gameweek_compSeason_label', 'teams_name'], right_on=['gameweek_compSeason_label', 'teams_name'])
df_games

,gameweek_compSeason_label,teams_name,formation_used_x,accurate_back_zone_pass_x,total_back_zone_pass_x,accurate_pass_x,touches_x,total_pass_x,backward_pass_x,successful_open_play_pass_x,open_play_pass_x,possession_percentage_x,leftside_pass_x,accurate_fwd_zone_pass_x,rightside_pass_x,total_long_balls_x,total_fwd_zone_pass_x,touches_in_opp_box_x,final_third_entries_x,crosses_18yard_x,successful_final_third_passes_x,accurate_cross_x,long_pass_own_to_opp_x,fwd_pass_x,total_cross_nocorner_x,total_cross_x,total_final_third_passes_x,long_pass_own_to_opp_success_x,poss_lost_all_x,poss_lost_ctrl_x,pen_area_entries_x,passes_left_x,accurate_long_balls_x,crosses_18yardplus_x,accurate_cross_nocorner_x,attempts_ibox_x,total_scoring_att_x,att_openplay_x,att_lf_target_x,att_bx_centre_x,ontarget_scoring_att_x,ball_recovery_x,att_lf_total_x,poss_won_att_3rd_x,att_ibox_target_x,ontarget_att_assist_x,blocked_pass_x,total_att_assist_x,duel_won_x,won_contest_x,att_sv_low_centre_x,successful_put_through_x,total_contest_x,att_assist_openplay_x,put_through_x,poss_won_mid_3rd_x,total_tackle_x,interception_x,interception_won_x,won_tackle_x,duel_lost_x,accurate_layoffs_x,passes_right_x,total_chipped_pass_x,total_layoffs_x,accurate_chipped_pass_x,aerial_lost_x,effective_clearance_x,lost_corners_x,total_launches_x,total_clearance_x,attempts_conceded_ibox_x,fk_foul_won_x,poss_won_def_3rd_x,saved_ibox_x,saves_x,diving_save_x,challenge_lost_x,dispossessed_x,unsuccessful_touch_x,goal_kicks_x,accurate_goal_kicks_x,attempts_conceded_obox_x,head_clearance_x,effective_head_clearance_x,fk_foul_lost_x,attempted_tackle_foul_x,aerial_won_x,total_yel_card_x,shot_off_target_x,total_throws_x,att_rf_total_x,accurate_throws_x,att_ibox_miss_x,offtarget_att_assist_x,saved_obox_x,outfielder_block_x,att_rf_target_x,corner_taken_x,won_corners_x,att_sv_high_centre_x,att_miss_right_x,att_corner_x,total_corners_intobox_x,att_hd_total_x,effective_blocked_cross_x,blocked_cross_x,subs_made_x,accurate_launches_x,fouled_final_third_x,att_obx_centre_x,attempts_obox_x,att_obox_target_x,keeper_throws_x,accurate_keeper_throws_x,total_through_ball_x,big_chance_missed_x,big_chance_created_x,att_miss_left_x,accurate_through_ball_x,blocked_scoring_att_x,att_ibox_blocked_x,total_flick_on_x,formation_used_y,open_play_pass_y,leftside_pass_y,accurate_goal_kicks_y,possession_percentage_y,poss_lost_all_y,goal_kicks_y,poss_lost_ctrl_y,total_pass_y,total_long_balls_y,total_back_zone_pass_y,total_chipped_pass_y,successful_open_play_pass_y,accurate_long_balls_y,accurate_back_zone_pass_y,rightside_pass_y,unsuccessful_touch_y,accurate_pass_y,accurate_chipped_pass_y,touches_y,attempts_conceded_ibox_y,saves_y,saved_ibox_y,ball_recovery_y,challenge_lost_y,backward_pass_y,successful_final_third_passes_y,interception_won_y,duel_lost_y,total_fwd_zone_pass_y,total_layoffs_y,total_final_third_passes_y,fwd_pass_y,accurate_layoffs_y,long_pass_own_to_opp_y,accurate_keeper_throws_y,accurate_fwd_zone_pass_y,interception_y,passes_right_y,passes_left_y,keeper_throws_y,long_pass_own_to_opp_success_y,put_through_y,accurate_throws_y,poss_won_def_3rd_y,final_third_entries_y,poss_won_mid_3rd_y,total_throws_y,dispossessed_y,aerial_won_y,duel_won_y,pen_area_entries_y,won_corners_y,touches_in_opp_box_y,corner_taken_y,fk_foul_lost_y,ontarget_scoring_att_y,poss_won_att_3rd_y,att_assist_openplay_y,big_chance_missed_y,total_scoring_att_y,att_bx_centre_y,total_corners_intobox_y,total_cross_y,total_att_assist_y,ontarget_att_assist_y,att_ibox_target_y,attempts_ibox_y,att_sv_high_centre_y,big_chance_created_y,att_openplay_y,att_hd_total_y,total_contest_y,won_contest_y,won_tackle_y,total_tackle_y,blocked_pass_y,successful_put_through_y,shot_off_target_y,attempts_obox_y,att_obx_centre_y,att_rf_total_y,total_cross_nocorner_y,total_through_ball_y,crosses_18yard_y,fk_foul_won_y,fouled_final_third_y,att_corner_y,aerial_lost_y,crosses_18yardplus_y,att_ibox_miss_y,offtarget_att_assist_y,att_miss_right_y,att_lf_total_y,total_launches_y,att_rf

In [33]:
feature_list_games = ['match_id', 'gameweek_gameweek', 'gameweek_compSeason_label', 'teams_team_1_name', 'teams_team_2_name', 'ground_name', 'team_1_hue']

In [34]:
#Датасет для бейзлайна
df[feature_list_games].to_csv(Path('data/base_line.csv').resolve(), index=False)
df.drop('ground_name', axis=1, inplace=True)

In [35]:
df['month'] = df['kickoff_label'].apply(lambda x: x.split()[2])
df['day_week'] = df['kickoff_label'].apply(lambda x: x.split()[0])
df['hour'] = df['kickoff_label'].apply(lambda x: x.split()[4].split(':')[0])

feature_list_games.extend(['month', 'day_week', 'hour', 'ground_id'])

In [36]:
df_club.head()

,season_id,club_name,club_abbr,club_id,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle,_attendance_count,_attendance_total,_attendance_average,_attendance_highest,_attendance_lowest,season
0,578,Arsenal,ARS,1.0,4792.0,372.0,41.0,109.0,69.0,13.0,8.0,6358.0,85.0,9.0,113.0,59.0,116.0,429.0,11041.0,7.0,26.0,266.0,299.0,285.0,132.0,119.0,74.0,253.0,112.0,225.0,25.0,113.0,41.0,43.0,9.0,11.0,15.0,70.0,11.0,1.0,5.0,4.0,7.0,16.0,1

In [37]:
df_club.drop(labels=['_attendance_count', '_attendance_total', '_attendance_average',	'_attendance_highest', '_attendance_lowest'], axis=1, inplace=True)

In [38]:
list_na = list(df_club.isna().sum()[df_club.isna().sum() > 0].index)

In [39]:
df_club[list_na] = df_club[list_na].fillna(df_club[df_club['season']!=2023][list_na].mean())

In [40]:
cols_mean = list(df_club.columns)[4:-1]
cols_mean = ['club_name'] + cols_mean

In [41]:
df_club_lag = df_club.drop(labels=['season_id', 'club_abbr', 'club_id'], axis=1)
df_club_lag['season'] = df_club_lag['season'] + 1
df_club_lag

,club_name,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle,season
0,Arsenal,4792.0,372.0,41.0,109.0,69.0,13.0,8.0,6358.0,85.0,9.0,113.0,59.0,116.0,429.0,11041.0,7.0,26.0,266.0,299.0,285.0,132.0,119.0,74.0,253.0,112.0,225.0,25.0,113.0,41.0,43.0,9.0,11.0,15.0,70.0,11.0,1.000000,5.0,4.0,7.0,16.0,10.0,11.0,29.0,1.0,8.0,57.0,66.0,40.0,81.0,6.0,61.0,13.0,34.0,142.0,28.0,15.0,10.0,33.0,31.0,56.0,4.0,36.0,16.0,112.0,4.0,272.0,6.0

In [42]:
df_num_mean = df_club[df_club['season']!=2023][cols_mean].groupby('club_name', as_index=False).mean()

In [43]:
df_num_mean.head()

,club_name,_accurate_back_zone_pass,_accurate_chipped_pass,_accurate_corners_intobox,_accurate_cross,_accurate_cross_nocorner,_accurate_flick_on,_accurate_freekick_cross,_accurate_fwd_zone_pass,_accurate_goal_kicks,_accurate_keeper_sweeper,_accurate_keeper_throws,_accurate_launches,_accurate_layoffs,_accurate_long_balls,_accurate_pass,_accurate_pull_back,_accurate_through_ball,_accurate_throws,_aerial_lost,_aerial_won,_attempted_tackle_foul,_attempts_conceded_ibox,_attempts_conceded_obox,_attempts_ibox,_attempts_obox,_att_assist_openplay,_att_assist_setplay,_att_bx_centre,_att_bx_left,_att_bx_right,_att_cmiss_high,_att_cmiss_left,_att_cmiss_right,_att_corner,_att_fastbreak,_att_goal_high_centre,_att_goal_high_left,_att_goal_high_right,_att_goal_low_centre,_att_goal_low_left,_att_goal_low_right,_att_hd_goal,_att_hd_miss,_att_hd_post,_att_hd_target,_att_hd_total,_att_ibox_blocked,_att_ibox_goal,_att_ibox_miss,_att_ibox_post,_att_ibox_target,_att_lf_goal,_att_lf_target,_att_lf_total,_att_miss_high,_att_miss_high_left,_att_miss_high_right,_att_miss_left,_att_miss_right,_att_obox_blocked,_att_obox_goal,_att_obox_miss,_att_obox_target,_att_obx_centre,_att_one_on_one,_att_openplay,_att_pen_goal,_att_post_left,_att_rf_goal,_att_rf_target,_att_rf_total,_att_setpiece,_att_sv_high_centre,_att_sv_high_left,_att_sv_high_right,_att_sv_low_centre,_att_sv_low_left,_att_sv_low_right,_backward_pass,_ball_recovery,_big_chance_created,_big_chance_missed,_big_chance_scored,_blocked_cross,_blocked_pass,_blocked_scoring_att,_challenge_lost,_clean_sheet,_clearance_off_line,_contentious_decision,_corner_taken,_crosses_18yard,_crosses_18yardplus,_defender_goals,_dispossessed,_diving_save,_draws,_duel_lost,_duel_won,_effective_blocked_cross,_effective_clearance,_effective_head_clearance,_error_lead_to_goal,_error_lead_to_shot,_final_third_entries,_first_half_goals,_fk_foul_lost,_fk_foul_won,_forward_goals,_fouled_final_third,_freekick_cross,_fwd_pass,_goals,_goals_conceded,_goals_conceded_ibox,_goals_conceded_obox,_goals_openplay,_goal_assist,_goal_assist_deadball,_goal_assist_intentional,_goal_assist_openplay,_goal_assist_setplay,_goal_fastbreak,_goal_kicks,_good_high_claim,_hand_ball,_head_clearance,_hit_woodwork,_interception,_interceptions_in_box,_interception_won,_keeper_throws,_last_man_tackle,_leftside_pass,_long_pass_own_to_opp,_long_pass_own_to_opp_success,_lost_corners,_midfielder_goals,_offtarget_att_assist,_ontarget_att_assist,_ontarget_scoring_att,_open_play_pass,_outfielder_block,_overrun,_own_goal_accrued,_passes_left,_passes_right,_penalty_conceded,_penalty_faced,_penalty_won,_pen_area_entries,_pen_goals_conceded,_possession_percentage,_poss_lost_all,_poss_lost_ctrl,_poss_won_att_3rd,_poss_won_def_3rd,_poss_won_mid_3rd,_post_scoring_att,_pts_dropped_winning_pos,_pts_gained_losing_pos,_punches,_put_through,_rightside_pass,_saved_ibox,_saved_obox,_saves,_second_yellow,_shield_ball_oop,_shot_fastbreak,_shot_off_target,_six_yard_block,_subs_goals,_subs_made,_successful_final_third_passes,_successful_open_play_pass,_successful_put_through,_total_att_assist,_total_back_zone_pass,_total_chipped_pass,_total_clearance,_total_contest,_total_corners_intobox,_total_cross,_total_cross_nocorner,_total_fastbreak,_total_final_third_passes,_total_flick_on,_total_fwd_zone_pass,_total_high_claim,_total_keeper_sweeper,_total_launches,_total_layoffs,_total_long_balls,_total_offside,_total_pass,_total_pull_back,_total_red_card,_total_scoring_att,_total_tackle,_total_through_ball,_total_throws,_total_yel_card,_touches,_touches_in_opp_box,_unsuccessful_touch,_wins,_won_contest,_won_corners,_won_tackle
0,Arsenal,7925.444444,693.777778,51.222222,150.333333,101.111111,63.444444,10.666667,9513.000000,154.333333,21.777778,196.444444,116.444444,396.777778,926.333333,17288.111111,18.444444,34.000000,646.333333,599.555556,574.333333,209.888889,271.000000,168.777778,327.788959,173.937107,321.777778,38.222222,195.333333,53.000000,48.222222,8.888889,18.555556,20.444444,68

In [44]:
df_lag = df_club_lag[df_club_lag['season']==2023]
df_lag.to_csv(Path('data/df_lag.csv').resolve(), index=False)

In [45]:
df_num_mean.to_csv(Path('data/club_mean.csv').resolve(), index=False)


In [46]:
df_1 = df[feature_list_games].merge(df_num_mean, left_on='teams_team_1_name', right_on='club_name')
df_1 = df_1.merge(df_num_mean, left_on='teams_team_2_name', right_on='club_name', suffixes=('_team_1', '_team_2'))
df_1.drop(['club_name_team_1', 'club_name_team_2'], axis=1, inplace=True)
df_1

KeyError: "['ground_name'] not in index"

In [ ]:
df_general = pd.merge(df_1, df_club_lag, left_on=['teams_team_1_name', 'gameweek_compSeason_label'], right_on=['club_name', 'season'], how='left')
df_general = pd.merge(df_general, df_club_lag, left_on=['teams_team_2_name', 'gameweek_compSeason_label'], right_on=['club_name', 'season'], how='left', suffixes=('_lag_team1', '_lag_team2'))

In [ ]:
df_general.sort_values(by=['match_id'], ascending=False , inplace=True)
df_general

match_id  gameweek_gameweek  gameweek_compSeason_label  \
0        93550                 23                       2023   
2354     93547                 23                       2023   
2677     93545                 23                       2023   
1626     93540                 22                       2023   
2113     93539                 22                       2023   
...        ...                ...                        ...   
117       9615                  1                       2014   
3323      9614                  1                       2014   
3177      9613                  1                       2014   
1199      9612                  1                       2014   
1517      9611                  1                       2014   

        teams_team_1_name  teams_team_2_name  team_1_hue month day_week hour  \
0        Sheffield United        Aston Villa           0   Feb      Sat   17   
2354              Everton  Tottenham Hotspur           2   Feb      Sat   12   
2677              Burnley             Fulham           2   Feb      Sat   15   
1626      Manchester City            Burnley           1   Jan      Wed   19   
2113            Liverpool            Chelsea           1   Jan      Wed   20   
...                   ...                ...         ...   ...      ...  ...   
117            Stoke City        Aston Villa           0   Aug      Sat   15   
3323  Queens Park Rangers          Hull City           0   Aug      Sat   15   
3177    Manchester United       Swansea City           0   Aug      Sat   12   
1199       Leicester City            Everton           2   Aug      Sat   15   
1517              Arsenal     Crystal Palace           1   Aug      Sat   17   

      ground_id  _accurate_back_zone_pass_team_1  \
0          2003                      4923.500000   
2354         31                      6304.777778   
2677         47                      4202.000000   
1626        457                      9053.555556   
2113        901                      8856.333333   
...         ...                              ...   
117         435                      5308.250000   
3323         46                      4135.000000   
3177         42                      7671.666667   
1199         49                      6395.666667   
1517         52                      7925.444444   

      _accurate_chipped_pass_team_1  _accurate_corners_intobox_team_1  \
0                        555.500000                         48.500000   
2354                     615.222222                         52.111111   
2677                     598.428571                         52.285714   
1626                     732.666667                         72.555556   
2113                     844.333333                         62.111111   
...                             ...                               ...   
117                      590.500000                         53.000000   
3323                     574.000000                         67.000000   
3177                     724.222222                         49.000000   
1199                     663.222222                         46.666667   
1517                     693.777778                         51.222222   

      _accurate_cross_team_1  _accurate_cross_nocorner_team_1  \
0                 161.500000                       114.500000   
2354              164.666667                       113.333333   
2677              171.428571                       120.000000   
1626              173.000000                       105.777778   
2113              180.222222                       119.888889   
...                      ...                              ...   
117               157.750000                       106.250000   
3323              182.000000                       116.000000   
3177              153.444444                       107.000000   
1199              146.777778                       102.777778   
1517              150.333333                       101.1111

In [ ]:
df_general.drop(['season_lag_team2', 'season_lag_team1', 'club_name_lag_team1', 'club_name_lag_team2'], axis=1, inplace=True)

In [ ]:
df_general.shape

(3372, 850)

In [ ]:
df_general.to_csv('data/df_res.csv', index=False)

In [ ]:
df_1.to_csv('data/df_1.csv', index=False)

Удалим признаки с высокой корреляцией

In [ ]:
df_1.iloc[:, 6:].corr()

,ground_id,_accurate_back_zone_pass_team_1,_accurate_chipped_pass_team_1,_accurate_corners_intobox_team_1,_accurate_cross_team_1,_accurate_cross_nocorner_team_1,_accurate_flick_on_team_1,_accurate_freekick_cross_team_1,_accurate_fwd_zone_pass_team_1,_accurate_goal_kicks_team_1,_accurate_keeper_sweeper_team_1,_accurate_keeper_throws_team_1,_accurate_launches_team_1,_accurate_layoffs_team_1,_accurate_long_balls_team_1,_accurate_pass_team_1,_accurate_pull_back_team_1,_accurate_through_ball_team_1,_accurate_throws_team_1,_aerial_lost_team_1,_aerial_won_team_1,_attempted_tackle_foul_team_1,_attempts_conceded_ibox_team_1,_attempts_conceded_obox_team_1,_attempts_ibox_team_1,_attempts_obox_team_1,_att_assist_openplay_team_1,_att_assist_setplay_team_1,_att_bx_centre_team_1,_att_bx_left_team_1,_att_bx_right_team_1,_att_cmiss_high_team_1,_att_cmiss_left_team_1,_att_cmiss_right_team_1,_att_corner_team_1,_att_fastbreak_team_1,_att_goal_high_centre_team_1,_att_goal_high_left_team_1,_att_goal_high_right_team_1,_att_goal_low_centre_team_1,_att_goal_low_left_team_1,_att_goal_low_right_team_1,_att_hd_goal_team_1,_att_hd_miss_team_1,_att_hd_post_team_1,_att_hd_target_team_1,_att_hd_total_team_1,_att_ibox_blocked_team_1,_att_ibox_goal_team_1,_att_ibox_miss_team_1,_att_ibox_post_team_1,_att_ibox_target_team_1,_att_lf_goal_team_1,_att_lf_target_team_1,_att_lf_total_team_1,_att_miss_high_team_1,_att_miss_high_left_team_1,_att_miss_high_right_team_1,_att_miss_left_team_1,_att_miss_right_team_1,_att_obox_blocked_team_1,_att_obox_goal_team_1,_att_obox_miss_team_1,_att_obox_target_team_1,_att_obx_centre_team_1,_att_one_on_one_team_1,_att_openplay_team_1,_att_pen_goal_team_1,_att_post_left_team_1,_att_rf_goal_team_1,_att_rf_target_team_1,_att_rf_total_team_1,_att_setpiece_team_1,_att_sv_high_centre_team_1,_att_sv_high_left_team_1,_att_sv_high_right_team_1,_att_sv_low_centre_team_1,_att_sv_low_left_team_1,_att_sv_low_right_team_1,_backward_pass_team_1,_ball_recovery_team_1,_big_chance_created_team_1,_big_chance_missed_team_1,_big_chance_scored_team_1,_blocked_cross_team_1,_blocked_pass_team_1,_blocked_scoring_att_team_1,_challenge_lost_team_1,_clean_sheet_team_1,_clearance_off_line_team_1,_contentious_decision_team_1,_corner_taken_team_1,_crosses_18yard_team_1,_crosses_18yardplus_team_1,_defender_goals_team_1,_dispossessed_team_1,_diving_save_team_1,_draws_team_1,_duel_lost_team_1,_duel_won_team_1,_effective_blocked_cross_team_1,_effective_clearance_team_1,_effective_head_clearance_team_1,_error_lead_to_goal_team_1,_error_lead_to_shot_team_1,_final_third_entries_team_1,_first_half_goals_team_1,_fk_foul_lost_team_1,_fk_foul_won_team_1,_forward_goals_team_1,_fouled_final_third_team_1,_freekick_cross_team_1,_fwd_pass_team_1,_goals_team_1,_goals_conceded_team_1,_goals_conceded_ibox_team_1,_goals_conceded_obox_team_1,_goals_openplay_team_1,_goal_assist_team_1,_goal_assist_deadball_team_1,_goal_assist_intentional_team_1,_goal_assist_openplay_team_1,_goal_assist_setplay_team_1,_goal_fastbreak_team_1,_goal_kicks_team_1,_good_high_claim_team_1,_hand_ball_team_1,_head_clearance_team_1,_hit_woodwork_team_1,_interception_team_1,_interceptions_in_box_team_1,_interception_won_team_1,_keeper_throws_team_1,_last_man_tackle_team_1,_leftside_pass_team_1,_long_pass_own_to_opp_team_1,_long_pass_own_to_opp_success_team_1,_lost_corners_team_1,_midfielder_goals_team_1,_offtarget_att_assist_team_1,_ontarget_att_assist_team_1,_ontarget_scoring_att_team_1,_open_play_pass_team_1,_outfielder_block_team_1,_overrun_team_1,_own_goal_accrued_team_1,_passes_left_team_1,_passes_right_team_1,_penalty_conceded_team_1,_penalty_faced_team_1,_penalty_won_team_1,_pen_area_entries_team_1,_pen_goals_conceded_team_1,_possession_percentage_team_1,_poss_lost_all_team_1,_poss_lost_ctrl_team_1,_poss_won_att_3rd_team_1,_poss_won_def_3rd_team_1,_poss_won_mid_3rd_team_1,_post_scoring_att_team_1,_pts_dropped_winning_pos_team_1,_pts_gained_losing_pos_team_1,_punches_team_1,_put_through_team_1,_rightside_pa

In [ ]:
def rm_high_corr_feat(df, thr, exc_col):
    features_to_remove = set()

    corr_matrix = df.iloc[:, exc_col:].corr()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > thr:
                colname = corr_matrix.columns[i]
                features_to_remove.add(colname)

    # Удаляем признаки из исходного датафрейма
    df = df.drop(columns=features_to_remove)

    # Рекурсивный вызов функции, если были удалены признаки
    if len(features_to_remove) > 0:
        return rm_high_corr_feat(df, thr, exc_col)
    else:
        return df

In [ ]:
df_rm_hcorr = rm_high_corr_feat(df_1, 0.8, 6)
df_rm_hcorr.shape

(3372, 170)

In [ ]:
df_rm_hcorr.head()

,match_id,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,team_1_hue,month,day_week,hour,ground_id,_accurate_back_zone_pass_team_1,_accurate_corners_intobox_team_1,_accurate_cross_team_1,_accurate_flick_on_team_1,_accurate_freekick_cross_team_1,_accurate_goal_kicks_team_1,_accurate_keeper_sweeper_team_1,_accurate_keeper_throws_team_1,_accurate_layoffs_team_1,_accurate_long_balls_team_1,_accurate_pull_back_team_1,_accurate_throws_team_1,_attempted_tackle_foul_team_1,_att_assist_setplay_team_1,_att_cmiss_high_team_1,_att_corner_team_1,_att_fastbreak_team_1,_att_goal_high_centre_team_1,_att_goal_high_right_team_1,_att_hd_goal_team_1,_att_hd_miss_team_1,_att_hd_post_team_1,_att_hd_target_team_1,_att_ibox_post_team_1,_att_miss_high_left_team_1,_att_miss_high_right_team_1,_att_obox_miss_team_1,_att_pen_goal_team_1,_att_post_left_team_1,_att_setpiece_team_1,_att_sv_high_centre_team_1,_att_sv_high_left_team_1,_att_sv_high_right_team_1,_ball_recovery_team_1,_blocked_pass_team_1,_challenge_lost_team_1,_clearance_off_line_team_1,_contentious_decision_team_1,_crosses_18yard_team_1,_crosses_18yardplus_team_1,_defender_goals_team_1,_dispossessed_team_1,_draws_team_1,_duel_lost_team_1,_duel_won_team_1,_error_lead_to_goal_team_1,_error_lead_to_shot_team_1,_final_third_entries_team_1,_fk_foul_won_team_1,_fouled_final_third_team_1,_freekick_cross_team_1,_goals_conceded_obox_team_1,_goal_assist_deadball_team_1,_goal_assist_setplay_team_1,_goal_fastbreak_team_1,_good_high_claim_team_1,_hand_ball_team_1,_interception_team_1,_last_man_tackle_team_1,_long_pass_own_to_opp_team_1,_overrun_team_1,_own_goal_accrued_team_1,_penalty_conceded_team_1,_poss_lost_all_team_1,_poss_won_def_3rd_team_1,_pts_dropped_winning_pos_team_1,_pts_gained_losing_pos_team_1,_punches_team_1,_put_through_team_1,_second_yellow_team_1,_shield_ball_oop_team_1,_subs_goals_team_1,_subs_made_team_1,_successful_put_through_team_1,_total_contest_team_1,_total_offside_team_1,_total_red_card_team_1,_total_tackle_team_1,_total_yel_card_team_1,_unsuccessful_touch_team_1,_accurate_back_zone_pass_team_2,_accurate_corners_intobox_team_2,_accurate_cross_team_2,_accurate_flick_on_team_2,_accurate_freekick_cross_team_2,_accurate_goal_kicks_team_2,_accurate_keeper_sweeper_team_2,_accurate_keeper_throws_team_2,_accurate_layoffs_team_2,_accurate_long_balls_team_2,_accurate_pull_back_team_2,_accurate_throws_team_2,_attempted_tackle_foul_team_2,_att_assist_setplay_team_2,_att_cmiss_high_team_2,_att_corner_team_2,_att_fastbreak_team_2,_att_goal_high_centre_team_2,_att_goal_high_right_team_2,_att_hd_goal_team_2,_att_hd_miss_team_2,_att_hd_post_team_2,_att_hd_target_team_2,_att_ibox_post_team_2,_att_miss_high_left_team_2,_att_miss_high_right_team_2,_att_obox_miss_team_2,_att_pen_goal_team_2,_att_post_left_team_2,_att_setpiece_team_2,_att_sv_high_centre_team_2,_att_sv_high_left_team_2,_att_sv_high_right_team_2,_ball_recovery_team_2,_blocked_pass_team_2,_challenge_lost_team_2,_clearance_off_line_team_2,_contentious_decision_team_2,_crosses_18yard_team_2,_crosses_18yardplus_team_2,_defender_goals_team_2,_dispossessed_team_2,_draws_team_2,_duel_lost_team_2,_duel_won_team_2,_error_lead_to_goal_team_2,_error_lead_to_shot_team_2,_final_third_entries_team_2,_fk_foul_won_team_2,_fouled_final_third_team_2,_freekick_cross_team_2,_goals_conceded_obox_team_2,_goal_assist_deadball_team_2,_goal_assist_setplay_team_2,_goal_fastbreak_team_2,_good_high_claim_team_2,_hand_ball_team_2,_interception_team_2,_last_man_tackle_team_2,_long_pass_own_to_opp_team_2,_overrun_team_2,_own_goal_accrued_team_2,_penalty_conceded_team_2,_poss_lost_all_team_2,_poss_won_def_3rd_team_2,_pts_dropped_winning_pos_team_2,_pts_gained_losing_pos_team_2,_punches_team_2,_put_through_team_2,_second_yellow_team_2,_shield_ball_oop_team_2,_subs_goals_team_2,_subs_made_team_2,_successful_put_through_team_2,_total_contest_team_2,_total_offside_team_2,_total_red_card_team_2,_total_tackle_team_2,_total_yel_card_team_2,_uns

In [ ]:
df_rm_hcorr.to_csv('data/df_rm_hcorr.csv', index=False)

## Prepare players

In [ ]:
player_team = '../../../parsers/official/data/player_team.csv'
df_player = '../../../EDA/official/eda_data/players_stat.csv'
df_goal = '../../../parsers/official/data/goalkippers.csv'


player_team = pd.read_csv(Path(player_team).resolve())
df_player = pd.read_csv(Path(df_player).resolve())
df_goal = pd.read_csv(Path(df_goal).resolve())

In [ ]:
player_team['team'].unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton and Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Fulham', 'Liverpool', 'Luton Town', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Nottingham Forest',
       'Sheffield United', 'Tottenham Hotspur', 'West Ham United',
       'Wolverhampton Wanderers', 'Leeds United', 'Leicester City',
       'Southampton', 'Norwich City', 'Watford', 'West Bromwich Albion',
       'Cardiff City', 'Huddersfield Town', 'Stoke City', 'Swansea City',
       'Hull City', 'Middlesbrough', 'Sunderland', 'Queens Park Rangers'],
      dtype=object)

In [ ]:
player_team.groupby(['team', 'season']).count()['player_id'].min()

19

In [ ]:
player_team.groupby(['team', 'season']).count()['player_id'].max()

61

In [ ]:
player_team

,season,team,player_id,position,name,appearances,height,weight,goals,assists,tackles,shots,keyPasses,cleanSheets
0,2023,Arsenal,284417,D,Ben White,21,186.0,76.0,1,1,25,7,18,6
1,2023,Arsenal,202648,D,Gabriel Magalhães,20,190.0,78.0,3,0,22,15,3,5
2,2023,Arsenal,313972,D,William Saliba,22,192.0,92.0,1,1,23,6,4,8
3,2023,Arsenal,337014,D,Jakub Kiwior,10,189.0,75.0,0,0,8,2,0,1
4,2023,Arsenal,288984,D,Takehiro Tomiyasu,13,187.0,84.0,1,1,11,8,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5264,2014,West Ham United,14145,F,Enner Valencia,32,177.0,74.0,4,3,30,72,18,2
5265,2014,West Ham United,6548,F,Elliot Lee,1,180.0,72.0,0,0,1,0,0,0
5266,2014,West Ham United,6535,M,Ravel Morrison,1,175.0,71.0,0,0,0,0,0,0
5267,2014,West Ham United,6545,F,Ricardo Vaz Tê,4,188.0,80.0,0,0,4,7,3,0


In [ ]:
#min amount defenders
player_team[player_team['position']=='D'].groupby(['team', 'season']).count()['player_id'].min()

7

In [ ]:
#Min amount forwards
player_team[player_team['position']=='F'].groupby(['team', 'season']).count()['player_id'].min()

3

In [ ]:
#Min amount midlefielders
player_team[player_team['position']=='M'].groupby(['team', 'season']).count()['player_id'].min()

6

Формируем датасет, где объект - команда за какой то сезон, в котором информация о игроках и их статистика. Игроков мы будем ранжировать, что бы было все одинаково, отбирать мы будем по количеству игр за сезон. Сначала будут идту 3 форварда, потом 6 мидлов и 7 защитников, потом вратарь и в конце менеджер.
Что бы не было утечки сдвигаем статистику на год назад

In [ ]:
player_team['season'].max()

2023

In [ ]:
player_team_t = player_team.copy()

In [ ]:
player_team_t[player_team_t['name']=='Ben White']

,season,team,player_id,position,name,appearances,height,weight,goals,assists,tackles,shots,keyPasses,cleanSheets
0,2023,Arsenal,284417,D,Ben White,21,186.0,76.0,1,1,25,7,18,6
443,2022,Arsenal,284417,D,Ben White,38,186.0,76.0,2,5,60,10,26,9
1023,2021,Arsenal,284417,D,Ben White,32,186.0,76.0,0,0,44,12,13,13
1801,2020,Brighton and Hove Albion,186780,D,Ben White,36,186.0,76.0,0,0,49,10,8,11


In [ ]:
player_team_t['season'] = player_team_t['season']+1
player_team_t[player_team_t['name']=='Ben White']

,season,team,player_id,position,name,appearances,height,weight,goals,assists,tackles,shots,keyPasses,cleanSheets
0,2024,Arsenal,284417,D,Ben White,21,186.0,76.0,1,1,25,7,18,6
443,2023,Arsenal,284417,D,Ben White,38,186.0,76.0,2,5,60,10,26,9
1023,2022,Arsenal,284417,D,Ben White,32,186.0,76.0,0,0,44,12,13,13
1801,2021,Brighton and Hove Albion,186780,D,Ben White,36,186.0,76.0,0,0,49,10,8,11


In [ ]:
player_team[player_team['name']=='Ben White']

,season,team,player_id,position,name,appearances,height,weight,goals,assists,tackles,shots,keyPasses,cleanSheets
0,2023,Arsenal,284417,D,Ben White,21,186.0,76.0,1,1,25,7,18,6
443,2022,Arsenal,284417,D,Ben White,38,186.0,76.0,2,5,60,10,26,9
1023,2021,Arsenal,284417,D,Ben White,32,186.0,76.0,0,0,44,12,13,13
1801,2020,Brighton and Hove Albion,186780,D,Ben White,36,186.0,76.0,0,0,49,10,8,11


In [ ]:
#Cдвигаем статистику на год
df_player_lag = pd.merge(player_team[['season', 'team', 'player_id', 'position', 'name']], player_team_t.drop(['player_id', 'position', 'team'], axis=1), on=['season', 'name'], how='left')
df_player_lag = df_player_lag[(df_player_lag['season'] != 2024) & (df_player_lag['season'] != 2014)]
df_player_lag.head()

,season,team,player_id,position,name,appearances,height,weight,goals,assists,tackles,shots,keyPasses,cleanSheets
0,2023,Arsenal,284417,D,Ben White,38.0,186.0,76.0,2.0,5.0,60.0,10.0,26.0,9.0
1,2023,Arsenal,202648,D,Gabriel Magalhães,38.0,190.0,78.0,3.0,0.0,48.0,32.0,5.0,14.0
2,2023,Arsenal,313972,D,William Saliba,27.0,192.0,92.0,2.0,1.0,34.0,6.0,6.0,11.0
3,2023,Arsenal,337014,D,Jakub Kiwior,7.0,189.0,75.0,1.0,0.0,7.0,1.0,1.0,1.0
4,2023,Arsenal,288984,D,Takehiro Tomiyasu,21.0,187.0,84.0,0.0,1.0,27.0,3.0,4.0,1.0


In [ ]:
def flatten_team(df, team_dict, col, type):
    
    df = df[col]
    for i in range(len(df)):

        for j in range(len(col)):
            team_dict[type + '_' + str(i) + '_' + col[j]].append(df.iloc[i, j])

    return team_dict
        

In [ ]:
team_dict = defaultdict(list)
col_players = ['name', 'appearances', 'height',	'weight', 'goals', 'assists', 'tackles', 'shots', 'keyPasses', 'cleanSheets']
col_gk = ['name', 'appearances', 'height',	'weight', 'saves', 'cleanSheets', 'goalsConceded']


for season in list(player_team['season'].unique()):
    for team in player_team['team'].unique():
        if player_team[(player_team['season']==season) & (player_team['team']==team)].shape[0] > 11:

            team_dict['season'].append(season)
            team_dict['team'].append(team)

            #players
            df_f = player_team[(player_team['season']==int(season)) & (player_team['team']==str(team)) & (player_team['position']=='F')].sort_values(by='appearances', ascending=False)[:3]
            df_m = player_team[(player_team['season']==int(season)) & (player_team['team']==team) & (player_team['position']=='M')].sort_values(by='appearances', ascending=False)[:6]
            df_d = player_team[(player_team['season']==int(season)) & (player_team['team']==team) & (player_team['position']=='D')].sort_values(by='appearances', ascending=False)[:7]

            team_dict = flatten_team(df_f, team_dict, col_players, 'F')
            team_dict = flatten_team(df_m, team_dict, col_players, 'M')
            team_dict = flatten_team(df_d, team_dict, col_players, 'D')

            #goalkeepers
            df_gk = df_goal[(df_goal['season']==int(season)) & (df_goal['team']==team)].sort_values(by='appearances', ascending=False)[:1]
            team_dict = flatten_team(df_gk, team_dict, col_gk, 'GK')

df_team_players = pd.DataFrame(team_dict)

Датасет, где результаты последних 10 матчей команды (лаги)

In [ ]:
df_res_home = df[['match_id', 'teams_team_1_name', 'team_1_hue']]
df_res_guest = df[['match_id', 'teams_team_2_name', 'team_1_hue']]
df_res_guest['team_1_hue'].replace({1:0, 0:1}, inplace=True)

df_res_home.rename(columns={'teams_team_1_name':'team', 'team_1_hue': 'result'}, inplace=True)
df_res_guest.rename(columns={'teams_team_2_name':'team', 'team_1_hue': 'result'}, inplace=True)

In [ ]:
print(df_res_home.shape, df_res_guest.shape)

(3643, 3) (3643, 3)


In [ ]:
df_res = pd.concat([df_res_home, df_res_guest], axis=0)
df_res.sort_values(by='match_id', inplace=True)
df_res.reset_index(drop=True, inplace=True)
df_res.shape

(7286, 3)

In [ ]:
df_res.head()

,match_id,team,result
0,9611,Crystal Palace,0
1,9611,Arsenal,1
2,9612,Everton,2
3,9612,Leicester City,2
4,9613,Manchester United,0


In [ ]:
team_lag = 6
df_team_lag = pd.DataFrame(columns=df_res.columns.tolist() + ['result_lag_' + str(i) for i in range(1, team_lag)])
for team in df_res['team'].unique():
    df_temp = df_res[df_res['team']==team]
    for i in range(1, team_lag):
        lag = df_temp['result'].shift(i)
        df_temp['result_lag_' + str(i)] = lag

    df_team_lag = pd.concat([df_team_lag, df_temp], axis=0)


In [ ]:
df_team_lag.drop('result', axis=1, inplace=True)

Датасет с лагами по результатам матчей между командами

In [ ]:
t = df[['match_id', 'teams_team_1_name', 'teams_team_2_name', 'team_1_hue']]

In [ ]:
#Датефрейм который агригирует результаты команды независимо от того, играла она дома или в гостях
df_game_team = pd.DataFrame(columns=t.columns)

for team_1 in t['teams_team_1_name'].unique():
    for team_2 in t['teams_team_2_name'].unique():
        df_temp_1 = t[(t['teams_team_1_name']==team_1) & (t['teams_team_2_name']==team_2)]
        df_temp_2 = t[(t['teams_team_1_name']==team_2) & (t['teams_team_2_name']==team_1)]

        df_temp_2['team_1_hue'].replace({1:0, 0:1}, inplace=True)
        df_temp_2.rename(columns={'teams_team_1_name':'teams_team_2_name', 'teams_team_2_name':'teams_team_1_name'}, inplace=True)

        df_temp = pd.concat([df_temp_1, df_temp_2], axis=0)

        df_game_team = pd.concat([df_game_team, df_temp], axis=0)

df_game_team.sort_values(by='match_id', inplace=True)                   

In [ ]:
df_game_team

,match_id,teams_team_1_name,teams_team_2_name,team_1_hue
3636,9611,Crystal Palace,Arsenal,0
3636,9611,Arsenal,Crystal Palace,1
3637,9612,Everton,Leicester City,2
3637,9612,Leicester City,Everton,2
3642,9613,Manchester United,Swansea City,0
...,...,...,...,...
4,93547,Everton,Tottenham Hotspur,2
3,93549,Luton Town,Newcastle United,2
3,93549,Newcastle United,Luton Town,2
0,93550,Aston Villa,Sheffield United,1


In [ ]:
n_lag = 3
df_game_lag = pd.DataFrame(columns=df_game_team.columns.tolist() + ['game_lag_' + str(i) for i in range(1, n_lag+1)])
for team_1 in df_game_team['teams_team_1_name'].unique():
    for team_2 in df_game_team['teams_team_2_name'].unique():
        df_temp = df_game_team[(df_game_team['teams_team_1_name']==team_1) & (df_game_team['teams_team_2_name']==team_2)]
        #Добавляем лаги
        if df_temp.shape[0] > 0:
            for i in range(1, n_lag+1):
                lag = df_temp['team_1_hue'].shift(i)

                df_temp['game_lag_' + str(i)] = lag
    
        df_game_lag = pd.concat([df_game_lag, df_temp], axis=0)

df_game_lag.sort_values(by='match_id', inplace=True)
df_game_lag.drop('team_1_hue', axis=1, inplace=True)

In [ ]:
df_game_lag

,match_id,teams_team_1_name,teams_team_2_name,game_lag_1,game_lag_2,game_lag_3
3636,9611,Crystal Palace,Arsenal,NaN,NaN,NaN
3636,9611,Arsenal,Crystal Palace,NaN,NaN,NaN
3637,9612,Everton,Leicester City,NaN,NaN,NaN
3637,9612,Leicester City,Everton,NaN,NaN,NaN
3642,9613,Manchester United,Swansea City,NaN,NaN,NaN
...,...,...,...,...,...,...
4,93547,Tottenham Hotspur,Everton,1,2,1
3,93549,Luton Town,Newcastle United,1,NaN,NaN
3,93549,Newcastle United,Luton Town,0,NaN,NaN
0,93550,Aston Villa,Sheffield United,2,0,1


In [ ]:
#Проверка
t[(t['teams_team_1_name']=='Luton Town') & (t['teams_team_2_name']=='Newcastle United') | (t['teams_team_1_name']=='Newcastle United') & (t['teams_team_2_name']=='Luton Town')]

,match_id,teams_team_1_name,teams_team_2_name,team_1_hue
3,93549,Newcastle United,Luton Town,2
48,93495,Luton Town,Newcastle United,1


Соединим датасеты
df_general  
df_team_players  
df_team_lag по каждой команде  
df_game_lag объединяем по двум командам  

In [ ]:
df_general = pd.merge(df_general, df_team_players, left_on=['teams_team_1_name', 'gameweek_compSeason_label'], right_on=['team', 'season'], how='left')
df_general = pd.merge(df_general, df_team_players, left_on=['teams_team_2_name', 'gameweek_compSeason_label'], right_on=['team', 'season'], how='left', suffixes=('_team_1', '_team_2'))

df_general.drop(['season_team_1', 'season_team_2', 'team_team_1', 'team_team_2'], axis=1, inplace=True)

In [ ]:
df_general = pd.merge(df_general, df_team_lag, how='left', left_on=['match_id', 'teams_team_1_name'], right_on=['match_id', 'team'])
df_general = pd.merge(df_general, df_team_lag, how='left', left_on=['match_id', 'teams_team_2_name'], right_on=['match_id', 'team'], suffixes=('_team_1', '_team_2'))

df_general.drop(['team_team_1', 'team_team_2'], axis=1, inplace=True)

In [ ]:
df_general = pd.merge(df_general, df_game_lag, how='left', on=['match_id', 'teams_team_1_name', 'teams_team_2_name'])

In [ ]:
df_general

match_id  gameweek_gameweek  gameweek_compSeason_label  \
0       93550                 23                       2023   
1       93547                 23                       2023   
2       93545                 23                       2023   
3       93540                 22                       2023   
4       93539                 22                       2023   
...       ...                ...                        ...   
3367     9615                  1                       2014   
3368     9614                  1                       2014   
3369     9613                  1                       2014   
3370     9612                  1                       2014   
3371     9611                  1                       2014   

        teams_team_1_name  teams_team_2_name  team_1_hue month day_week hour  \
0        Sheffield United        Aston Villa           0   Feb      Sat   17   
1                 Everton  Tottenham Hotspur           2   Feb      Sat   12   
2                 Burnley             Fulham           2   Feb      Sat   15   
3         Manchester City            Burnley           1   Jan      Wed   19   
4               Liverpool            Chelsea           1   Jan      Wed   20   
...                   ...                ...         ...   ...      ...  ...   
3367           Stoke City        Aston Villa           0   Aug      Sat   15   
3368  Queens Park Rangers          Hull City           0   Aug      Sat   15   
3369    Manchester United       Swansea City           0   Aug      Sat   12   
3370       Leicester City            Everton           2   Aug      Sat   15   
3371              Arsenal     Crystal Palace           1   Aug      Sat   17   

      ground_id  _accurate_back_zone_pass_team_1  \
0          2003                      4923.500000   
1            31                      6304.777778   
2            47                      4202.000000   
3           457                      9053.555556   
4           901                      8856.333333   
...         ...                              ...   
3367        435                      5308.250000   
3368         46                      4135.000000   
3369         42                      7671.666667   
3370         49                      6395.666667   
3371         52                      7925.444444   

      _accurate_chipped_pass_team_1  _accurate_corners_intobox_team_1  \
0                        555.500000                         48.500000   
1                        615.222222                         52.111111   
2                        598.428571                         52.285714   
3                        732.666667                         72.555556   
4                        844.333333                         62.111111   
...                             ...                               ...   
3367                     590.500000                         53.000000   
3368                     574.000000                         67.000000   
3369                     724.222222                         49.000000   
3370                     663.222222                         46.666667   
3371                     693.777778                         51.222222   

      _accurate_cross_team_1  _accurate_cross_nocorner_team_1  \
0                 161.500000                       114.500000   
1                 164.666667                       113.333333   
2                 171.428571                       120.000000   
3                 173.000000                       105.777778   
4                 180.222222                       119.888889   
...                      ...                              ...   
3367              157.750000                       106.250000   
3368              182.000000                       116.000000   
3369              153.444444                       107.000000   
3370              146.777778                       102.777778   
3371              150.333333                       101.111111   

    

In [ ]:
df_general.sort_values(by=['match_id'], inplace=True)

In [ ]:
df_general.to_csv(Path('data/df_general.csv').resolve(), index=False)